### OpenAI Assistant Agent

This notebook shows how to consume [`OpenAI Assistant API`](https://platform.openai.com/docs/assistants/overview) using `OpenAIAssistantAgent` class.

In [1]:
// Install dependencies
#i "nuget:https://www.myget.org/F/agentchat/api/v3/index.json"

#r "nuget:AgentChat.OpenAI,*-*"

Restore sources https://www.myget.org/F/agentchat/api/v3/index.json Installed Packages AgentChat.OpenAI, 0.4.2

#### Create `OpenAIAssistantAgent`

You can either create a new `OpenAIAssistantAgent`, which create a new `OpenAI Assistant` on the server, or load an existing `OpenAI Assistant`.

In [2]:
using AgentChat.OpenAI;
using Azure.AI.OpenAI;

var apiKey = Environment.GetEnvironmentVariable("OPENAI_API_KEY") ?? throw new Exception("OPENAI_API_KEY not set");
var openAIClient = new OpenAIClient(apiKey);
var assistant = await OpenAIAssistantAgent.CreateAsync(
    openAIClient,
    name: "assistant",
    roleInformation: "You are a helpful AI assistant",
    description: "A helpful AI assistant that will answer your questions",
    model: "gpt-3.5-turbo",
    useCodeInterpreter: false,
    useRetrieval: false);

// retrieve existing assistant
var id = assistant.ID;
var assistantObject = await openAIClient.RetrieveAssistantAsync(id);
var retrievedAssistant = new OpenAIAssistantAgent(openAIClient, assistantObject);

#### Interact with `OpenAIAssistantAgent`

In [3]:
using AgentChat;
var reply = await assistant.SendMessageAsync("Hello, how are you?");
reply.PrettyPrintMessage();

Message from assistant
--------------------
Hello! I'm an AI assistant, so I don't have emotions, but I'm here to help you with any questions or tasks you have. How can I assist you today?

--------------------



#### use code interpreter to resolve task via coding

You can enable `OpenAI code_interpreter` tool in `OpenAIAssistantAgent` by setting `useCodeInterpreter` to true

In [4]:
var question = "what's 33th of fibonacci minus 23th of fibonacci"; //  3,495,921
// agent without code interpreter
var reply = await assistant.SendMessageAsync(question);
reply.PrettyPrintMessage();

// agent with code interpreter
var assistantWithCodeInterpreter = await OpenAIAssistantAgent.CreateAsync(
    openAIClient,
    name: "assistant",
    roleInformation: "You are a helpful AI assistant",
    model: "gpt-3.5-turbo",
    description: "A helpful AI assistant that will answer your questions",
    useCodeInterpreter: true,
    useRetrieval: false);
reply = await assistantWithCodeInterpreter.SendMessageAsync(question);
reply.PrettyPrintMessage();

Message from assistant
--------------------
To calculate the difference between the 33rd and 23rd Fibonacci numbers, we can use the formula for the nth Fibonacci number: 

F(n) = F(n-1) + F(n-2)

First, let's find the 33rd Fibonacci number:

F(33) = F(32) + F(31)

Now, let's find the 32nd Fibonacci number:

F(32) = F(31) + F(30)

Now, let's find the 31st Fibonacci number:

F(31) = F(30) + F(29)

Next, let's find the 30th Fibonacci number:

F(30) = F(29) + F(28)

Now, let's find the 29th Fibonacci number:

F(29) = F(28) + F(27)

Now, let's find the 28th Fibonacci number:

F(28) = F(27) + F(26)

Now, let's find the 27th Fibonacci number:

F(27) = F(26) + F(25)

Now, let's find the 26th Fibonacci number:

F(26) = F(25) + F(24)

Now, let's find the 25th Fibonacci number:

F(25) = F(24) + F(23)

Next, let's find the 24th Fibonacci number:

F(24) = F(23) + F(22)

Now, let's find the 23rd Fibonacci number:

F(23) = F(22) + F(21)

Finally, substituting all the values back into our original equ

#### Custom function call
You can enable Function calling in `OpenAIAssistantAgent` by passing a `FunctionMap` when creating `OpenAIAssistantAgent`

In [5]:
using System.Text.Json;

var guessNumberFunctionDefinition = new FunctionDefinition
	{
		Name = @"GuessNumber",
        Description = """
Guess integer number between 1 and 10
""",
        Parameters = BinaryData.FromObjectAsJson(new
        {
            Type = "object",
            Properties = new
			{
			    number = new
			    {
				    Type = @"number",
				    Description = @"number to guess",
			    },
            },
            Required = new []
			{
			    "number",
			},
        },
        new JsonSerializerOptions
		{
			PropertyNamingPolicy = JsonNamingPolicy.CamelCase,
		})
    };

var guessNumberFunc = async (string args) => {
	var obj = JsonSerializer.Deserialize<JsonElement>(args);
	var number = obj.GetProperty("number").GetInt32();
	Console.WriteLine($"number: {number}");
	if (number < 1 || number > 10)
	{
		Console.WriteLine("Number must be between 1 and 10");
		return "Number must be between 1 and 10";
	}

	if (number == 7)
	{
		Console.WriteLine("You guessed it!");
		return "You guessed it!";
	}

	if (number < 7)
	{
		Console.WriteLine("Too low");
		return "Too low";
	}

	Console.WriteLine("Too high");
	return "Too high";
};

var assistant = await OpenAIAssistantAgent.CreateAsync(
	client: openAIClient,
	name: "assistant",
	roleInformation: "You are a helpful AI assistant",
	model: "gpt-3.5-turbo",
	description: "A helpful AI assistant that will answer your questions",
	useCodeInterpreter: false,
	useRetrieval: false,
	functionMaps: new Dictionary<FunctionDefinition, Func<string, Task<string>>>
	{
		{ guessNumberFunctionDefinition, guessNumberFunc },
	}
);

var reply = await assistant.SendMessageAsync("guess a integer number between 1 and 10, and tell me the number once you guess it");

reply.PrettyPrintMessage();


number: 5
Too low
number: 8
Too high
number: 6
Too low
number: 7
You guessed it!
Message from assistant
--------------------
The number is 7.

--------------------



#### Remove `OpenAIAssistantAgent`

You can remove the `OpenAI Assistant` from portal by calling `RemoveAssistantAsync`

In [7]:
// remove assistant
await openAIClient.RemoveAssistantAsync(assistant.ID);

// remove assistant with code interpreter
await openAIClient.RemoveAssistantAsync(assistantWithCodeInterpreter.ID);

Error: System.Exception: fail to remove assistant asst_k8UeTkDrSot05cxNFjyiwGbX
   at AgentChat.OpenAI.OpenAIClientExtension.RemoveAssistantAsync(OpenAIClient client, String assistantId, Nullable`1 ct)
   at Submission#8.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)